<a href="https://colab.research.google.com/github/bish-ai/Bishal.py/blob/main/christmAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from itertools import permutations
from collections import defaultdict
import time

class Box:
    def __init__(self, id, x, y, z):
        self.id = id
        self.dims = sorted([x, y, z], reverse=True)
        self.x, self.y, self.z = self.dims
        self.volume = x * y * z
        self.rotations = self._get_rotations(x, y, z)

    def _get_rotations(self, x, y, z):
        unique = set()
        for perm in permutations([x, y, z]):
            unique.add(perm)
        return list(unique)

class Container:
    def __init__(self, x, y, z):
        self.x, self.y, self.z = x, y, z
        self.volume = x * y * z
        self.spaces = [(0, 0, 0, x, y, z)]
        self.placed = []

    def can_fit(self, box, rotation, space):
        sx, sy, sz, sw, sh, sd = space
        bx, by, bz = rotation
        return bx <= sw and by <= sh and bz <= sd

    def place_box(self, box, rotation, space_idx):
        space = self.spaces[space_idx]
        sx, sy, sz, sw, sh, sd = space
        bx, by, bz = rotation

        self.placed.append({
            'box_id': box.id,
            'x': sx, 'y': sy, 'z': sz,
            'dx': bx, 'dy': by, 'dz': bz
        })

        del self.spaces[space_idx]

        # Generate new spaces using guillotine cuts
        new_spaces = []

        # Space above
        if bz < sd:
            new_spaces.append((sx, sy, sz + bz, bx, by, sd - bz))

        # Space to the right
        if bx < sw:
            new_spaces.append((sx + bx, sy, sz, sw - bx, sh, sd))

        # Space in front
        if by < sh:
            new_spaces.append((sx, sy + by, sz, bx, sh - by, sd))

        self.spaces.extend(new_spaces)
        self.spaces.sort(key=lambda s: (s[2], s[1], s[0]))  # Sort by z, y, x

    def get_fill_rate(self):
        used = sum(p['dx'] * p['dy'] * p['dz'] for p in self.placed)
        return used / self.volume

class AdvancedPacker:
    def __init__(self, container_dims):
        self.container_dims = container_dims
        self.best_solution = None
        self.best_score = 0

    def pack_boxes(self, boxes, time_limit=300):
        start = time.time()

        # Try multiple strategies
        strategies = [
            self._pack_volume_sorted,
            self._pack_dimension_sorted,
            self._pack_hybrid,
            self._pack_layer_by_layer
        ]

        for strategy in strategies:
            if time.time() - start > time_limit:
                break

            solution = strategy(boxes.copy())
            score = self._evaluate_solution(solution)

            if score > self.best_score:
                self.best_score = score
                self.best_solution = solution

        return self.best_solution

    def _pack_volume_sorted(self, boxes):
        boxes.sort(key=lambda b: b.volume, reverse=True)
        return self._greedy_pack(boxes)

    def _pack_dimension_sorted(self, boxes):
        boxes.sort(key=lambda b: (b.x, b.y, b.z), reverse=True)
        return self._greedy_pack(boxes)

    def _pack_hybrid(self, boxes):
        # Sort by volume/surface area ratio
        boxes.sort(key=lambda b: b.volume / (2*(b.x*b.y + b.y*b.z + b.z*b.x)), reverse=True)
        return self._greedy_pack(boxes)

    def _pack_layer_by_layer(self, boxes):
        container = Container(*self.container_dims)
        boxes.sort(key=lambda b: b.z, reverse=True)

        current_layer_height = 0
        layer_boxes = []

        for box in boxes:
            placed = False

            for rot in box.rotations:
                for i, space in enumerate(container.spaces):
                    if container.can_fit(box, rot, space):
                        # Prefer placing in current layer
                        if space[2] == current_layer_height or not layer_boxes:
                            container.place_box(box, rot, i)
                            layer_boxes.append(box)
                            placed = True
                            break
                if placed:
                    break

            if not placed:
                # Start new layer
                current_layer_height = max([p['z'] + p['dz'] for p in container.placed] or [0])
                layer_boxes = []

                for rot in box.rotations:
                    for i, space in enumerate(container.spaces):
                        if container.can_fit(box, rot, space):
                            container.place_box(box, rot, i)
                            layer_boxes.append(box)
                            break
                    if layer_boxes and layer_boxes[-1] == box:
                        break

        return container

    def _greedy_pack(self, boxes):
        container = Container(*self.container_dims)

        for box in boxes:
            best_space = None
            best_rotation = None
            best_score = float('inf')

            for i, space in enumerate(container.spaces):
                for rot in box.rotations:
                    if container.can_fit(box, rot, space):
                        # Score based on corner distance and wasted space
                        waste = (space[3] - rot[0]) * (space[4] - rot[1]) * (space[5] - rot[2])
                        corner_dist = space[0] + space[1] + space[2]
                        score = waste * 0.6 + corner_dist * 0.4

                        if score < best_score:
                            best_score = score
                            best_space = i
                            best_rotation = rot

            if best_space is not None:
                container.place_box(box, best_rotation, best_space)

        return container

    def _evaluate_solution(self, container):
        return container.get_fill_rate()

# Main execution
def solve_challenge(boxes_df, container_dims=(10, 10, 10)):
    """
    boxes_df: DataFrame with columns ['id', 'x', 'y', 'z']
    container_dims: tuple of (x, y, z) dimensions
    """
    boxes = [Box(row['id'], row['x'], row['y'], row['z'])
             for _, row in boxes_df.iterrows()]

    packer = AdvancedPacker(container_dims)
    solution = packer.pack_boxes(boxes, time_limit=300)

    # Create submission format
    results = pd.DataFrame(solution.placed)

    print(f"Fill Rate: {solution.get_fill_rate():.4f}")
    print(f"Boxes Packed: {len(solution.placed)}/{len(boxes)}")

    return results

# Example usage:
# df = pd.read_csv('boxes.csv')
# solution = solve_challenge(df, container_dims=(100, 100, 100))
# solution.to_csv('submission.csv', index=False)